In [3]:
%%capture
# Install the latest Tensorflow version.
!pip install -q tensorflow_text
!pip install -q simpleneighbors[annoy]
!pip install -q nltk
!pip install -q tqdm
!pip3 install tensorflow_text>=2.0.0rc0

In [4]:
import numpy as np
import pandas as pd
import tensorflow_hub as hub
import tensorflow as tf
import tensorflow_text
import seaborn as sn

In [6]:
data = pd.read_excel('WHO_FAQ.xlsx')
data.head()

,Context,Answer
0,What is a coronavirus?,Coronaviruses are a large family of viruses wh...
1,What is a coronavirus?,"In humans, several coronaviruses are known to ..."
2,What is COVID-19?,COVID-19 is the infectious disease caused by t...
3,What are the symptoms of COVID-19?,The most common symptoms of COVID-19 are fever...
4,What are the symptoms of COVID-19?,Some people become infected but don’t develop ...


In [7]:
data.shape

(85, 2)

In [8]:
module = hub.load('https://tfhub.dev/google/universal-sentence-encoder-qa/3')

In [9]:
# model
data.iloc[1]["Answer"]
response_encoding = module.signatures['response_encoder'](
    input=tf.constant(data.Answer),
    context=tf.constant(data.Context)
)['outputs']
# question_encoding = module.signatures['question_encoder'](
#     tf.constant(data.Context)
# )

In [10]:
test_question = ["What about pregnant women?"]

question_encoding = module.signatures['question_encoder'](
    tf.constant(test_question)
)['outputs']


In [11]:
response = np.argmax(np.inner(question_encoding, response_encoding),axis=1)
response_s = np.argsort(np.inner(question_encoding, response_encoding),axis=1)

In [14]:
data.iloc[response[0]].Answer

'Pregnant women should take the same precautions to avoid COVID-19 infection as other people.'